In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/projetyasmine/folds.csv
/kaggle/input/projetyasmine/Train.csv
/kaggle/input/projetyasmine/Test.csv
/kaggle/input/projetyasmine/Images/id_dkbp68fw0g.jpg
/kaggle/input/projetyasmine/Images/id_cz3jnoimvm.jpg
/kaggle/input/projetyasmine/Images/id_rvu3jjuh34_rgn.jpg
/kaggle/input/projetyasmine/Images/id_akzsic5cnz_rgn.jpg
/kaggle/input/projetyasmine/Images/id_004wknd7qd.jpg
/kaggle/input/projetyasmine/Images/id_nqmwafm3jo.jpg
/kaggle/input/projetyasmine/Images/id_wwo1nsomhj_rgn.jpg
/kaggle/input/projetyasmine/Images/id_z0hq8jpog2.jpg
/kaggle/input/projetyasmine/Images/id_d9lk3ojg11_rgn.jpg
/kaggle/input/projetyasmine/Images/id_uj6bcc6774_rgn.jpg
/kaggle/input/projetyasmine/Images/id_falxfhtpdw_rgn.jpg
/kaggle/input/projetyasmine/Images/id_nvepupqkuu_rgn.jpg
/kaggle/input/projetyasmine/Images/id_mjhv8d9dvy.jpg
/kaggle/input/projetyasmine/Images/id_aw9y79nutx_rgn.jpg
/kaggle/input/projetyasmine/Images/id_lfz4f0u3hl_rgn.jpg
/kaggle/input/projetyasmine/Images/id_us09j03fde_rgn.jpg

In [ ]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
df = pd.read_csv(r"D:\DATA_SCIENCE\MASTER_2\ProjectData354\Train.csv")
encoder = LabelEncoder()
df['score'] = encoder.fit_transform(df['Label'])
df = df.reset_index()
df.drop(df[df.index %2 != 0].index, inplace=True)
df = df[['Image_id', 'score']].reset_index(drop = True)
df

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (_, valid_index) in enumerate(kf.split(df, df['score'])):
    df.loc[valid_index, 'fold'] = int(n)

df['fold'] = df['fold'].astype(int)
df

In [ ]:
test.head()
df[df['fold'] == 4][df[df['fold'] == 4]['score'] == 0]

In [ ]:
df.to_csv(r'folds.csv')

In [ ]:
test = pd.read_csv(r"D:\DATA_SCIENCE\MASTER_2\ProjectData354\Test.csv")

In [ ]:
test.head()

In [13]:
pip install efficientnet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import  Image
import torch
from torch import nn, optim
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet





In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

In [15]:
def seet_everyting(seed = 42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True 
seet_everyting()

In [67]:
class Dataset(Dataset):
    def __init__(self, image, file, mode, fold, h, w, transform = None):
        self.image     = image
        self.w         = w 
        self.h         = h
        self.transform = transform
        if mode == 'train': self.file = pd.read_csv(file)[pd.read_csv(file)['fold'] != fold].reset_index(drop = True)
        else              : self.file = pd.read_csv(file)[pd.read_csv(file)['fold'] == fold].reset_index(drop = True)
    
    def __len__(self):
        return self.file.shape[0]
    
    def __getitem__(self, index):
        img_path  = os.path.join(self.image, self.file['Image_id'][index])

        image     = np.array(Image.open(img_path).convert('RGB'))

        if self.transform is not None:
            augmentations = self.transform(image = image)
            image = augmentations['image']

        y = self.file['score'][index]
        if y == 0 : y = [1,0,0]
        if y == 1 : y = [0,1,0]
        else      : y = [0,0,1]

        return image, torch.tensor(y)

In [68]:
LR = 3e-4
BS = 32
NE = 6
H = 224
W = 192

IMG = r"/kaggle/input/projetyasmine/Images"
FILE = r"/kaggle/input/projetyasmine/folds.csv"


def get_loaders(image, file, fold, h, w, bs, train_transform, val_transform, pin_memory = True):
    train_ds = Dataset(image = image, file = file, mode = 'train', fold = fold, h = h, w =w, transform = train_transform)
    train_loader = DataLoader(train_ds, batch_size=bs, pin_memory = pin_memory, shuffle=True)

    val_ds = Dataset(image = image, file = file, mode = 'val', fold = fold, h = h, w =w, transform = val_transform)
    val_loader = DataLoader(val_ds, batch_size=bs, pin_memory = pin_memory, shuffle=True)

    return train_loader, val_loader
normalize = A.Normalize(
    mean = [0.485, 0.456, 0.406],
    std  = [0.229, 0.224, 0.255],
    max_pixel_value = 255.0
)

train_transform = A.Compose(
    [
        A.Resize(width= W, height= H), 
        A.HorizontalFlip(p = 0.6),
        A.VerticalFlip(p = 0.4),
        A.Rotate(limit=40),
        normalize,
        ToTensorV2()
    ]
)

val_transform = A.Compose(
    [
        A.Resize(width= W, height= H), 
        normalize,
        ToTensorV2()
    ]
)

In [69]:

# Définir le modèle ResNet-18
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18 = models.resnet18(pretrained=True)
        in_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Linear(in_features, 3)

    def forward(self, x):
        return self.resnet18(x)


In [85]:
def check_acc(loader, model, loss_fn):
    correct_predictions = 0
    total_samples = 0
    total_loss = 0

    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device).to(torch.float32)
            y = y.to(device).to(torch.float32)  # Assurez-vous que y est de type long
            
            scores       = model(x)
            total_loss   = loss_fn(scores, y)
        average_loss = total_loss / len(loader)
        print("Average Loss:", average_loss)
    
    model.train()

    return average_loss


In [90]:
# Définir les paramètres
device = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 3e-4
BS = 32
NE = 4
H = 224
W = 192



# Boucle d'entraînement sur les plis
for fold in range(5):
    print('--------------------------- Fold: ' + str(fold))
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=LR)

    train_loader, val_loader = get_loaders(
        IMG, FILE, fold, H, W, BS, train_transform, val_transform
    )
    
    best_loss = 1.0
    for epoch in range(NE):
        print(' ------------------------------ Epoch: ' + str(epoch))
        train_fn(train_loader, model, optimizer, loss_fn, scaler)
        loss = check_acc(val_loader, model, loss_fn)

        # Sauvegarder le modèle si la perte de validation est améliorée
        if loss < best_loss:
            best_loss = loss
            checkpoint = {
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict()
            }
            save_checkpoint(checkpoint, filename=f'rice_disease_fold{fold}.pth.tar')


--------------------------- Fold: 0


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


 ------------------------------ Epoch: 0


100%|██████████| 17/17 [00:01<00:00,  9.48it/s]


Average Loss: tensor(0.0223, device='cuda:0')
--> saving checkpoint
 ------------------------------ Epoch: 1


...

Average Loss: tensor(0.0227, device='cuda:0')
--------------------------- Fold: 4
 ------------------------------ Epoch: 0


100%|██████████| 17/17 [00:01<00:00,  9.42it/s]


Average Loss: tensor(0.0321, device='cuda:0')
--> saving checkpoint
 ------------------------------ Epoch: 1


100%|██████████| 17/17 [00:01<00:00,  9.13it/s]


Average Loss: tensor(0.0286, device='cuda:0')
--> saving checkpoint


In [91]:

# Charger le fichier Test.csv
test_df = pd.read_csv("/kaggle/input/test-set/test_set.csv")
# Créer un DataFrame pour stocker les prédictions
predictions_df = pd.DataFrame(columns=['Image_id', 'blast', 'brown', 'healthy'])

# Prédire pour chaque image dans le fichier Test.csv
for index, row in test_df.iterrows():
    Id = row['Image_id']


    image_path = os.path.join('/kaggle/input/projetyasmine/Images', Id)
    
# Charger l'image de test et appliquer les transformations
    image = Image.open(image_path).convert('RGB')
    transformed_data = val_transform(image=np.array(image))

# Extraire l'image transformée du dictionnaire
    image = transformed_data["image"]

    image = image.unsqueeze(0)
    
    # Déplacer l'image sur cuda si nécessaire
    image = image.to('cuda')

# Effectuer la prédiction
    with torch.no_grad():
        model.eval()
        output = model(image)

    # Appliquer la fonction softmax pour obtenir les probabilités
    probs = torch.nn.functional.softmax(output, dim=1)

    # Récupérer les probabilités pour chaque classe
    prob_blast = probs[0, 0].item()
    prob_brown = probs[0, 1].item()
    prob_healthy = probs[0, 2].item()

    # Créer un dictionnaire avec les valeurs à ajouter
    new_row = {
        'Image_id': Id,
        'blast': prob_blast,
        'brown': prob_brown,
        'healthy': prob_healthy
    }

    # Ajouter la nouvelle ligne au DataFrame
    predictions_df.loc[len(predictions_df)] = new_row

# Sauvegarder les prédictions dans un fichier CSV
predictions_df.to_csv('/kaggle/working/soumission_Eff0.csv', index=False)
